# Phase 2 Variable Tracking Test

This notebook tests the variable tracking functionality that would be used by the Jupyter extension.

**Instructions:**
1. Run each cell in order
2. The extension would track these variables and show them in right-click context menus
3. Check the console output to see what variables are detected

In [ ]:
# Cell 1: Import libraries (no variables created)
import numpy as np
import sys
import os

# Add the extension to path for testing
sys.path.insert(0, 'jupyter_shm_extension')

print("✅ Libraries imported")

In [ ]:
# Cell 2: Create some basic variables
data = np.random.randn(1000, 4)
fs = 1000.0
channels = ["X", "Y", "Z", "RX"]

print(f"✅ Created variables:")
print(f"   data: shape {data.shape}, type {type(data).__name__}")
print(f"   fs: {fs}, type {type(fs).__name__}")
print(f"   channels: {channels}, type {type(channels).__name__}")

In [ ]:
# Cell 3: Simulate SHM function calls (tuple unpacking)
# These would normally be shmtools functions, but we'll simulate them

def mock_ar_model_shm(data, order=15):
    """Mock AR model function"""
    features = np.random.randn(data.shape[0]-order, order*data.shape[1])
    model = {'order': order, 'coefficients': np.random.randn(order)}
    return features, model

features, model = mock_ar_model_shm(data, order=15)

print(f"✅ Created SHM variables:")
print(f"   features: shape {features.shape}, type {type(features).__name__}")
print(f"   model: keys {list(model.keys()) if isinstance(model, dict) else 'N/A'}, type {type(model).__name__}")

In [ ]:
# Cell 4: More complex assignments with parentheses
(scores, loadings) = np.random.randn(features.shape[0], 2), np.random.randn(features.shape[1], 2)

print(f"✅ Created more variables:")
print(f"   scores: shape {scores.shape}, type {type(scores).__name__}")
print(f"   loadings: shape {loadings.shape}, type {type(loadings).__name__}")

In [ ]:
# Cell 5: Test the variable parsing functionality
from handlers import SHMVariableInspectionHandler

# Simulate notebook cell data that the extension would see
mock_cells = [
    {
        'cell_type': 'code',
        'source': '''# Cell 1: Import libraries\nimport numpy as np\nimport sys\nimport os'''
    },
    {
        'cell_type': 'code', 
        'source': '''data = np.random.randn(1000, 4)\nfs = 1000.0\nchannels = ["X", "Y", "Z", "RX"]'''
    },
    {
        'cell_type': 'code',
        'source': '''features, model = mock_ar_model_shm(data, order=15)'''
    },
    {
        'cell_type': 'code',
        'source': '''(scores, loadings) = np.random.randn(features.shape[0], 2), np.random.randn(features.shape[1], 2)'''
    }
]

# Create a mock handler to test parsing
class MockHandler:
    def _parse_notebook_variables(self, cells):
        import re
        variables = []
        
        for cell_index, cell in enumerate(cells):
            if cell.get('cell_type') != 'code':
                continue
                
            code = cell.get('source', '')
            cell_variables = self._extract_variables_from_code(code, cell_index)
            variables.extend(cell_variables)
        
        return variables
    
    def _extract_variables_from_code(self, code, cell_index):
        import re
        variables = []
        lines = code.split('\n')
        
        for line_index, line in enumerate(lines):
            line = line.strip()
            
            if line.startswith('#') or not line:
                continue
            
            patterns = [
                r'^([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)',
                r'^([a-zA-Z_][a-zA-Z0-9_,\s]*)\s*=\s*(.+)',
                r'^\(([a-zA-Z_][a-zA-Z0-9_,\s]*)\)\s*=\s*(.+)'
            ]
            
            for pattern in patterns:
                match = re.match(pattern, line)
                if match:
                    left_side = match.group(1).strip()
                    right_side = match.group(2).strip()
                    
                    if ',' in left_side:
                        var_names = [v.strip().replace('(', '').replace(')', '') for v in left_side.split(',')]
                        for var_name in var_names:
                            if var_name:
                                variables.append({
                                    'name': var_name,
                                    'type': self._infer_type_from_expression(right_side),
                                    'source': f'Cell {cell_index + 1}',
                                    'cell_index': cell_index,
                                    'expression': right_side
                                })
                    else:
                        variables.append({
                            'name': left_side,
                            'type': self._infer_type_from_expression(right_side),
                            'source': f'Cell {cell_index + 1}',
                            'cell_index': cell_index,
                            'expression': right_side
                        })
                    break
        
        return variables
    
    def _infer_type_from_expression(self, expression):
        import re
        expression = expression.split('#')[0].strip()
        
        if 'shmtools.' in expression:
            if any(func in expression for func in ['ar_model', 'pca', 'mahalanobis']):
                return 'tuple'
        
        if 'np.' in expression or 'numpy.' in expression:
            if any(func in expression for func in ['.array', '.zeros', '.ones', '.randn', '.random']):
                return 'numpy.ndarray'
        
        if 'mock_ar_model_shm' in expression:
            return 'tuple'
        
        if re.match(r'^\d+$', expression):
            return 'int'
        if re.match(r'^\d+\.\d+$', expression):
            return 'float'
        if expression.startswith('"') or expression.startswith("'"):
            return 'str'
        if expression.startswith('[') and expression.endswith(']'):
            return 'list'
        if ',' in expression and not expression.startswith('['):
            return 'tuple'
        
        return 'unknown'

# Test the parsing
handler = MockHandler()
parsed_vars = handler._parse_notebook_variables(mock_cells)

print("🔍 Variable parsing results (what the extension would see):")
print(f"Found {len(parsed_vars)} variables:")
for var in parsed_vars:
    print(f"  ✓ {var['name']} ({var['type']}) from {var['source']}")
    
print("\n📝 This demonstrates what would appear in the right-click context menu!")

## What Phase 2 Enables

When the full extension is installed, you would:

1. **See a dropdown** in the notebook toolbar with "SHM Functions"
2. **Right-click on any parameter** in a function call (like `data=None`)
3. **Get a context menu** showing all the variables from previous cells:
   - `data (numpy.ndarray) - Cell 2`
   - `fs (float) - Cell 2` 
   - `channels (list) - Cell 2`
   - `features (tuple) - Cell 3`
   - `model (dict) - Cell 3`
   - `scores (tuple) - Cell 4`
   - `loadings (tuple) - Cell 4`
4. **Click on a variable** to automatically replace the parameter value

The parsing works in real-time as you type and execute cells!

In [ ]:
# Cell 6: Test what would happen in a real SHM workflow
print("📊 Simulating SHM workflow parameter selection:")
print("\n1. User types: result = shmtools.pca_shm(data=None, order=15)")
print("2. User right-clicks on 'None' after 'data='")
print("3. Context menu shows available variables:")

# Show what variables would be available at this point
available_vars = [var for var in parsed_vars if var['cell_index'] < 5]  # Before this cell
for var in available_vars:
    compatible = "✅" if var['type'] in ['numpy.ndarray', 'tuple'] else "⚠️"
    print(f"   {compatible} {var['name']} ({var['type']}) from {var['source']}")

print("\n4. User clicks on 'data' → code becomes: result = shmtools.pca_shm(data=data, order=15)")
print("\n🎯 This is exactly what Phase 2 enables!")